## Import Data and Library

In [20]:
import os
os.chdir(r'C:\Users\nadda\Desktop\KU\01204314-65-Statistics-for-Computer-Engineering-Applications\final_project\part2.1')
os.getcwd()

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('omw-1.4')  # optional แต่ช่วยให้ lemmatizer ทำงานดีขึ้น

import re
import numpy as np
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

from gensim.models import Word2Vec


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nadda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nadda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\nadda\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nadda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nadda\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [21]:
file = open("africa.txt", "r", encoding='utf-8')
africa = file.read()
file.close()

file = open("asia.txt", "r", encoding='utf-8')
asia = file.read()
file.close()

file = open("europe.txt", "r", encoding='utf-8')
europe = file.read()
file.close()

file = open("latin_america.txt", "r", encoding='utf-8')
latin_america = file.read()
file.close()

## Text Processing

In [22]:
# Tokenize
africa_token = word_tokenize(africa)
asia_token = word_tokenize(asia)
europe_token = word_tokenize(europe)
latin_america_token = word_tokenize(latin_america)

In [23]:
# Remove punctuation
africa_token = [re.sub(r'[^\w\s]', '', token) for token in africa_token if re.sub(r'[^\w\s]', '', token)]
asia_token = [re.sub(r'[^\w\s]', '', token) for token in asia_token if re.sub(r'[^\w\s]', '', token)]
europe_token = [re.sub(r'[^\w\s]', '', token) for token in europe_token if re.sub(r'[^\w\s]', '', token)]
latin_america_token = [re.sub(r'[^\w\s]', '', token) for token in latin_america_token if re.sub(r'[^\w\s]', '', token)]

In [24]:
# Remove stop words
stop_words = set(stopwords.words('english'))
africa_token = [token for token in africa_token if token.lower() not in stop_words]
asia_token = [token for token in asia_token if token.lower() not in stop_words]
europe_token = [token for token in europe_token if token.lower() not in stop_words]
latin_america_token = [token for token in latin_america_token if token.lower() not in stop_words]

In [25]:
# POS tagging
africa_pos = pos_tag(africa_token)
asia_pos = pos_tag(asia_token)
europe_pos = pos_tag(europe_token)
latin_america_pos = pos_tag(latin_america_token)

In [26]:
# Lemmatization
lemmatizer = WordNetLemmatizer()

# แปลง POS tag จาก nltk → wordnet format
def convert_pos(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default เป็นคำนาม
    
def lemmatize_and_lowercase(word_pos_list):
    result = []
    for word, pos in word_pos_list:
        wn_pos = convert_pos(pos)
        lemma = lemmatizer.lemmatize(word, wn_pos)
        if wn_pos != wordnet.NOUN:
            lemma = lemma.lower()
        result.append(lemma)
    return result

africa_lemmatized = lemmatize_and_lowercase(africa_pos)
asia_lemmatized = lemmatize_and_lowercase(asia_pos)
europe_lemmatized = lemmatize_and_lowercase(europe_pos)
latin_america_lemmatized = lemmatize_and_lowercase(latin_america_pos)

## Text Analysis

In [27]:
from gensim.models import Word2Vec

# รวมทุก region: [[word1, word2, ...], [word1, word2, ...], ...]
all_tokens = [africa_lemmatized, asia_lemmatized, europe_lemmatized, latin_america_lemmatized]

# Train Word2Vec
model = Word2Vec(
    sentences=all_tokens,
    vector_size=100,   # ขนาดเวกเตอร์
    window=5,          # ขอบเขตคำรอบข้าง
    min_count=1,       # เอาทุกคำ แม้เจอน้อย
    sg=1,              # ใช้ Skip-gram
    epochs=100,        # รอบการ train
    seed=42            # reproducibility
)

# หา 30 คำที่ใกล้คำว่า "gender"
print('30 Most related Words to "gender"')
model.wv.most_similar('gender', topn=30)

30 Most related Words to "gender"


[('equality', 0.7441256046295166),
 ('reach', 0.7101626396179199),
 ('improvement', 0.6999989151954651),
 ('room', 0.6813947558403015),
 ('still', 0.681369960308075),
 ('vary', 0.6641895771026611),
 ('encourage', 0.6611723303794861),
 ('gaps', 0.6609336137771606),
 ('basis', 0.6480785608291626),
 ('adopt', 0.6382238864898682),
 ('much', 0.6311314105987549),
 ('full', 0.6296327710151672),
 ('commitment', 0.6286253333091736),
 ('towards', 0.6273753643035889),
 ('fact', 0.624669075012207),
 ('Despite', 0.6235596537590027),
 ('analyze', 0.6224499940872192),
 ('persist', 0.6219894289970398),
 ('trend', 0.6170200705528259),
 ('inequality', 0.6161401867866516),
 ('underestimate', 0.6156435012817383),
 ('widely', 0.6152939796447754),
 ('member', 0.6131133437156677),
 ('Data', 0.6126700639724731),
 ('substantive', 0.6092368364334106),
 ('dimension', 0.607306957244873),
 ('Reduce', 0.606299638748169),
 ('association', 0.6027237772941589),
 ('expansion', 0.5994047522544861),
 ('priority', 0.59615

In [28]:
# Train Word2Vec
model = Word2Vec(
    sentences=all_tokens,
    vector_size=100,   # ขนาดเวกเตอร์
    window=5,          # ขอบเขตคำรอบข้าง
    min_count=1,       # เอาทุกคำ แม้เจอน้อย
    sg=1,              # ใช้ Skip-gram
    epochs=100,        # รอบการ train
    seed=42            # reproducibility
)

# หา 30 คำที่ใกล้คำว่า "equality"
print('30 Most related Words to "equality"')
model.wv.most_similar('equality', topn=30)

30 Most related Words to "equality"


[('gaps', 0.7567237019538879),
 ('gender', 0.7441256642341614),
 ('substantive', 0.7253093123435974),
 ('Data', 0.69869065284729),
 ('commitment', 0.6740468144416809),
 ('towards', 0.6629829406738281),
 ('reach', 0.6616057753562927),
 ('dimension', 0.6502516269683838),
 ('adopt', 0.6367909908294678),
 ('improvement', 0.635064423084259),
 ('Incentives', 0.6318537592887878),
 ('analyze', 0.6291241645812988),
 ('full', 0.6269788146018982),
 ('Reduce', 0.6158241629600525),
 ('commit', 0.610556423664093),
 ('conduct', 0.6088600158691406),
 ('entity', 0.6078664660453796),
 ('various', 0.6077552437782288),
 ('adoption', 0.6054598689079285),
 ('room', 0.6028990149497986),
 ('framework', 0.5994222164154053),
 ('encourage', 0.5982713103294373),
 ('however', 0.5981284976005554),
 ('mainstreaming', 0.5964041948318481),
 ('member', 0.5925891995429993),
 ('expansion', 0.5868939757347107),
 ('governance', 0.5859586596488953),
 ('empower', 0.5858578681945801),
 ('code', 0.5839506983757019),
 ('still',